In [25]:
import numpy as np
import time
import matplotlib.pyplot as plt
import random, math
import sys
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.spatial.distance import cdist


In [26]:
def fourier_series(x, coefficients):
    #x = np.array(x)
    #min_x = np.min(x)
    #max_x = np.max(x)
    #L = max_x - min_x
    result = coefficients[0]  # Initialize with the constant term a0
    num_terms = (len(coefficients) - 1) // 2
    for m in range(1, num_terms + 1):
        a_m = coefficients[2*m - 1]
        b_m = coefficients[2*m]
        result += a_m * np.cos(m *(x)) + b_m * np.sin(m *(x))
    return result


In [27]:
def fourier_series_derivative(x, coefficients):
    #x = np.array(x)
    #min_x, max_x = (np.min(x), np.max(x))
    #L = max_x - min_x
    result = 0
    num_terms = (len(coefficients) - 1) // 2
    for m in range(1, num_terms + 1):
        a_m = coefficients[2*m - 1]
        b_m = coefficients[2*m]
        result -= m * a_m * np.sin(m * np.pi * (x)) 
        result += m * b_m * np.cos(m * np.pi * (x))
    return result

In [28]:
# Define the right-hand side of the differential equation
def rhs(x):
    return np.piecewise(x, [x >= 0, x < 0], [1, 0])

In [29]:
def objective_function(coefficients):
    # Numerically integrate the integral term
    integral_term = np.zeros_like(x_values)
    for i, x in enumerate(x_values):
        integral_term[i], _ = quad(lambda t: fourier_series(t, coefficients), 0, x)
    
    # Calculate the difference between LHS and RHS
    difference = fourier_series_derivative(x_values, coefficients) + 2 * fourier_series(x_values, coefficients) + 5 * integral_term - rhs(x_values)
    
    # Return the squared integral of the difference
    return np.trapz(difference**2, x_values)

In [30]:
def exact_solution(t):
    # Example: Exact solution 
    return 0.5*np.exp(-t)*np.sin(2*t)

In [21]:
def red_deer_algorithm(objective_function, bounds, population_size=50, max_iter=100):
    # Red Deer Algorithm
    ############################### Parameters ####################################
    #   num_deers: number of red deers                                            #
    #   max_iter: maximum number of generations                                   #
    #   obj_function: the function to maximize while doing feature selection      #
    ###############################################################################
    
    # Initialize population
    num_deers = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(population_size, bounds.shape[0]))
    fitness = np.zeros(len(num_deers))
    accuracy = np.zeros(len(num_deers))
    Leader_agent = np.zeros((1, bounds.shape[0]))
    Leader_fitness = float("-inf")
    Leader_accuracy = float("-inf")
    
    # initializing parameters
    UB = 5 # Upper bound
    LB = -5 # Lower bound
    gamma = 0.5 # Fraction of total number of males who are chosen as commanders
    alpha = 0.2 # Fraction of total number of hinds in a harem who mate with the commander of their harem
    beta = 0.1 # Fraction of total number of hinds in a harem who mate with the commander of a different harem
    
    # start timer
    start_time = time.time()     #  You can use to output the current time
    
    # main loop
    while iter_no <= max_iter:
        # (1) roaring of male deer
        for male in range(len(males)):
            r1 = np.random.random() # r1 is a random number in [0, 1]
            r2 = np.random.random() # r2 is a random number in [0, 1]
            r3 = np.random.random() # r3 is a random number in [0, 1]
            new_male = males[male].copy()
            if r3 >= 0.5:                                    # Eq. (3)
                new_male += r1 * (((UB - LB) * r2) + LB)
            else:
                new_male -= r1 * (((UB - LB) * r2) + LB)
            # Update the male at index i with the new values
            males[male] = new_male
        # print(males)
        
        # (2) selection of male commanders and stags
        num_coms = int(len(males) * gamma) # Eq. (4)
        num_stags = int(len(males)) - num_coms # Eq. (5)
        
        coms = males[:num_coms,:]
        stags = males[num_coms:,:]
        
        # (3) fight between male commanders and stags
        for i in range(num_coms):
            chosen_com = coms[i].copy()
            chosen_stag = random.choice(stags)
            r1 = np.random.random()
            r2 = np.random.random()
            new_male_1 = (chosen_com + chosen_stag) / 2 + r1 * (((UB - LB) * r2) + LB) # Eq. (6)
            new_male_2 = (chosen_com + chosen_stag) / 2 - r1 * (((UB - LB) * r2) + LB) # Eq. (7)
            
            # Update the position
            fitness = np.zeros(4)
            fitness[0] = objective_function(chosen_com)
            fitness[1] = objective_function(chosen_stag)
            fitness[2] = objective_function(new_male_1)
            fitness[3] = objective_function(new_male_2)

            bestfit = np.min(fitness)
            if fitness[0] < fitness[1] and fitness[1] == bestfit:
                coms[i] = chosen_stag.copy()
            elif fitness[0] < fitness[2] and fitness[2] == bestfit:
                coms[i] = new_male_1.copy()
            elif fitness[0] < fitness[3] and fitness[3] == bestfit:
                coms[i] = new_male_2.copy()
                new_population.append(list(coms[i].copy()))
                
        # (4) formation of harems
        norm = np.linalg.norm(fitness)
        normal_fit = fitness / norm
        total = np.sum(normal_fit)
        power = normal_fit / total # Eq. (9)
        num_harems = [int(x * len(females)) for x in power] # Eq.(10)
        max_harem_size = np.max(num_harems)
            
        # Create an empty numpy array for harems
        harem = np.empty((len(num_harems), max_harem_size), dtype=object)   
        random.shuffle(females)
        itr = 0
        for i in range(num_coms):
            harem_size = num_harems[i]
            for j in range(harem_size):
                harem[i][j] = females[itr]
                itr += 1
        
        # (5) mating of commander with hinds in his harem
        num_harem_mate = [int(x * alpha) for x in num_harems] # Eq. (11)
        population_pool = list(initial_population)
        for i in range(num_coms):
            random.shuffle(females[i])
            for j in range(num_harem_mate[i]):
                r = np.random.random() # r is a random number in [0, 1]
                offspring = (coms[i] + harem[i][j]) / 2 + (UB - LB) * r # Eq. (12)
                new_population.append(list(offspring.copy()))
                
                # select K 
                if num_coms > 1:
                # mating of commander with hinds in another harem
                    k = i 
                    while k == i:
                        k = random.choice(range(num_coms))
                        num_mate = int(num_harems[k] * beta) # Eq. (13)
                        np.random.shuffle(harem[k])
                        for j in range(num_mate):
                            r = np.random.random() # r is a random number in [0, 1]
                            offspring = (coms[i] + harem[k][j]) / 2 + (UB - LB) * r
                            new_population.append(list(offspring.copy()))
                            
            
        # (6) mating of stag with nearest hind
        for stag in stags:
            dist = np.zeros(len(females))
            for i in range(len(females)):
                dist[i] = math.sqrt(np.sum((stag-females[i])*(stag-females[i])))
                min_dist = np.min(dist)
                for i in range(len(females)):
                    distance = math.sqrt(np.sum((stag-females[i])*(stag-females[i]))) # Eq. (14)
                    if(distance == min_dist):
                        r = np.random.random() # r is a random number in [0, 1]
                        offspring = (stag + females[i])/2 + (UB - LB) * r       # Eq. (+12)
                        new_population.append(list(offspring.copy()))
                        
                        break
            
        # selection of the next generation
        while len(new_population) < population_size:
            r = np.random.randint(0, len(new_population))
            new_population.append(new_population[r])
        
        new_population = new_population[:population_size]
        
        # Calculate fitness for each individual
        fitness_scores = [(individual, objective_function(np.array(individual))) for individual in new_population]
        # Sort the population based on fitness scores (ascending order)
        sorted_population = sorted(fitness_scores, key=lambda x: x[1])

        # Print the sorted population
        for individual, fitness in sorted_population:
            print("Individual:", individual, "Fitness:", fitness)
        
        sorted_population = sorted_population

        
        
        # stop timer
        end_time = time.time()
        exec_time = end_time - start_time
        
        # update attributes of solution
        best_solution = sorted_population
        #best_fitness = fitness[indices]
        return best_solution


In [31]:
def red_deer_algorithm(objective_function, bounds, population_size=50, max_iter=100):
    # Red Deer Algorithm
    ############################### Parameters ####################################
    #   num_deers: number of red deers                                            #
    #   max_iter: maximum number of generations                                   #
    #   obj_function: the function to maximize while doing feature selection      #
    ###############################################################################
    
    # Initialize population
    num_deers = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(population_size, bounds.shape[0]))
    fitness = np.zeros(len(num_deers))
    accuracy = np.zeros(len(num_deers))
    Leader_agent = np.zeros((1, bounds.shape[0]))
    Leader_fitness = float("-inf")
    Leader_accuracy = float("-inf")
    
    # initializing parameters
    UB = 5 # Upper bound
    LB = -5 # Lower bound
    gamma = 0.5 # Fraction of total number of males who are chosen as commanders
    alpha = 0.2 # Fraction of total number of hinds in a harem who mate with the commander of their harem
    beta = 0.1 # Fraction of total number of hinds in a harem who mate with the commander of a different harem
    
    # start timer
    start_time = time.time()     #  You can use to output the current time
    
    # main loop
    iter_no =1
    while iter_no <= max_iter:
        for j in range(population_size):
            num_males = np.array([int(0.25 * num) for num in num_deers[j]]) 
            num_hinds = num_deers - num_males
            males = num_deers[:num_males,:]
            hinds = num_deers[num_males:,:]
            
            # (1) roaring of male deer
            for i in range(num_males):
                r1 = np.random.random() # r1 is a random number in [0, 1]
                r2 = np.random.random() # r2 is a random number in [0, 1]
                r3 = np.random.random() # r3 is a random number in [0, 1]
                new_male = males[i].copy()
                if r3 >= 0.5:                                    # Eq. (3)
                    new_male += r1 * (((UB - LB) * r2) + LB)
                else:
                    new_male -= r1 * (((UB - LB) * r2) + LB)
            
            # (2) selection of male commanders and stags
            num_coms = int(num_males * gamma) # Eq. (4)
            num_stags = num_males - num_coms # Eq. (5)

            coms = males[:num_coms,:]
            stags = males[num_coms:,:]
            
            # (3) fight between male commanders and stags
            for i in range(num_coms):
                chosen_com = coms[i].copy()
                chosen_stag = random.choice(stags)
                r1 = np.random.random()
                r2 = np.random.random()
                new_male_1 = (chosen_com + chosen_stag) / 2 + r1 * (((UB - LB) * r2) + LB) # Eq. (6)
                new_male_2 = (chosen_com + chosen_stag) / 2 - r1 * (((UB - LB) * r2) + LB) # Eq. (7)
            
                # Update the position
                fitness = np.zeros(4)
                fitness[0] = objective_function(chosen_com)
                fitness[1] = objective_function(chosen_stag)
                fitness[2] = objective_function(new_male_1)
                fitness[3] = objective_function(new_male_2)

                bestfit = np.max(fitness)
                if fitness[0] < fitness[1] and fitness[1] == bestfit:
                    coms[i] = chosen_stag.copy()
                elif fitness[0] < fitness[2] and fitness[2] == bestfit:
                    coms[i] = new_male_1.copy()
                elif fitness[0] < fitness[3] and fitness[3] == bestfit:
                    coms[i] = new_male_2.copy()
                
            # (4) formation of harems
            norm = np.linalg.norm(fitness)
            normal_fit = fitness / norm
            total = np.sum(normal_fit)
            power = normal_fit / total # Eq. (9)
            num_harems = [int(x * num_hinds) for x in power] # Eq.(10)
            max_harem_size = np.max(num_harems)
            harem = np.empty(shape=(num_coms, max_harem_size, num_features))
            random.shuffle(hinds)
            itr = 0
            for i in range(num_coms):
                harem_size = num_harems[i]
                for j in range(harem_size):
                    harem[i][j] = hinds[itr]
                    itr += 1
            
            # (5) mating of commander with hinds in his harem
            num_harem_mate = [int(x * alpha) for x in num_harems] # Eq. (11)
            population_pool = list(deer)
            for i in range(num_coms):
                random.shuffle(harem[i])
                for j in range(num_harem_mate[i]):
                    r = np.random.random() # r is a random number in [0, 1]
                    offspring = (coms[i] + harem[i][j]) / 2 + (UB - LB) * r # Eq. (12)


                    # select K 
                    if num_coms > 1:
                        # mating of commander with hinds in another harem
                        k = i 
                        while k == i:
                            k = random.choice(range(num_coms))

                        num_mate = int(num_harems[k] * beta) # Eq. (13)

                        np.random.shuffle(harem[k])
                        for j in range(num_mate):
                            r = np.random.random() # r is a random number in [0, 1]
                            offspring = (coms[i] + harem[k][j]) / 2 + (UB - LB) * r 
            
            # (6) mating of stag with nearest hind
            for stag in stags:
                dist = np.zeros(num_hinds)
                for i in range(num_hinds):
                    dist[i] = math.sqrt(np.sum((stag-hinds[i])*(stag-hinds[i])))
                min_dist = np.min(dist)
                for i in range(num_hinds):
                    distance = math.sqrt(np.sum((stag-hinds[i])*(stag-hinds[i]))) # Eq. (14)
                    if(distance == min_dist):
                        r = np.random.random() # r is a random number in [0, 1]
                        offspring = (stag + hinds[i])/2 + (UB - LB) * r       # Eq. (+12)
            
            # selection of the next generation
            population_pool = np.array(population_pool)            
            maximum = sum([f for f in fitness])
            selection_probs = [f/maximum for f in fitness]
            indices = np.random.choice(len(population_pool), size=num_deers, replace=True, p=selection_probs)          
            deer = population_pool[indices]
            
            # update final information
            display(deer, fitness, agent_name)
            if fitness[0] > Leader_fitness:
                Leader_agent = deer[0].copy()
                Leader_fitness = fitness[0].copy()
        # stop timer
        end_time = time.time()
        exec_time = end_time - start_time
        
        # update attributes of solution
        solution.best_fitness = Leader_fitness
        solution.final_population = deer
        solution.final_fitness = fitness
        solution.final_accuracy = accuracy
        solution.execution_time = exec_time
        return solution


In [32]:
# Bounds for the coefficients of the Fourier series
bounds = np.array([[-1, 1]] * 6)  # Example: 10 cosine terms and 10 sine terms

In [33]:
# Generate x values
x_values = np.linspace(0, np.pi, 100)

In [34]:
# Perform optimization using Red Deer Algorithm
best_solution, best_cost = red_deer_algorithm(objective_function, bounds, population_size=50, max_iter=100)

TypeError: only integer scalar arrays can be converted to a scalar index